### Spark homework 2 Movie Recommendation System
in this note book we use data from MovieLens to construct a movie recommendation system for users.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as sns
import math
import os

In [3]:
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [5]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
# Read data files into the script
movies = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [7]:
# preview movie table
movies.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [8]:
# preview ratings table
ratings.show(5)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

In [9]:
# preview links table
links.show(5)

+-------+-------+------+
movieId| imdbId|tmdbId|
+-------+-------+------+
 1|0114709| 862|
 2|0113497| 8844|
 3|0113228| 15602|
 4|0114885| 31357|
 5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows

In [10]:
# preview tags table
tags.show(5)

+------+-------+---------------+----------+
userId|movieId| tag| timestamp|
+------+-------+---------------+----------+
 2| 60756| funny|1445714994|
 2| 60756|Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

In [11]:
# find out minimum number of ratings a user give out
tmp1 = ratings.groupBy("userID").count().toPandas()['count'].min()
# find out minimum number of ratings received by users
tmp2 = ratings.groupby("userID").count().toPandas()['count'].min()

print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 20

In [12]:
# find out how many movies are rated by users proportionally
tmp1 = sum(ratings.groupBy("movieId").count().toPandas()['count'] == 1)
tmp2 = ratings.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user

### Spark SQL and OLAP

In [14]:
# create temporary table
movies.registerTempTable("movies")
ratings.registerTempTable("ratings")
links.registerTempTable("links")
tags.registerTempTable("tags")

### Q1: The number of users

In [16]:
%sql 
select count(distinct userID) as user_number 
from ratings

user_number
610


### Q2: The number of Movies

In [18]:
%sql 
select count(*) as movie_number 
from movies 

movie_number
9742


### Q3: How many movies are rated by users? List movies not rated before

In [20]:
%sql 
select 
count(distinct title) as movie_rated 
from 
(
select * 
from ratings r join movies m on r.movieID = m.movieID
)

movie_rated
9719


In [21]:
%sql 
select title, genres 
from movies m 
left join ratings r on m.movieID = r.movieID 
where rating is null 
order by 1

title,genres
"Browning Version, The (1951)",Drama
Call Northside 777 (1948),Crime|Drama|Film-Noir
Chalet Girl (2011),Comedy|Romance
"Chosen, The (1981)",Drama
"Color of Paradise, The (Rang-e khoda) (1999)",Drama
For All Mankind (1989),Documentary
I Know Where I'm Going! (1945),Drama|Romance|War
In the Realms of the Unreal (2004),Animation|Documentary
"Innocents, The (1961)",Drama|Horror|Thriller
Mutiny on the Bounty (1962),Adventure|Drama|Romance


### Q4: List Movie Genres

In [23]:
%sql 
select distinct genres 
from movies

genres
Comedy|Horror|Thriller
Adventure|Sci-Fi|Thriller
Action|Adventure|Drama|Fantasy
Action|Drama|Horror
Action|Animation|Comedy|Sci-Fi
Animation|Children|Drama|Musical|Romance
Action|Adventure|Drama
Adventure|Sci-Fi
Documentary|Musical|IMAX
Adventure|Children|Fantasy|Sci-Fi|Thriller


From above result, we can see that one movie can have multiple genres. Hence, distinct genres need to be separated from each other by "|" (separate string by column)

In [25]:
%sql 
select distinct col as genres 
from 
(
select explode(split(genres, '[|]')) 
from movies 
) 
order by 1

genres
(no genres listed)
Action
Adventure
Animation
Children
Comedy
Crime
Documentary
Drama
Fantasy


### Q5: Movie for Each Category

For each movie genre, count how many movies are tagged in it.

In [28]:
%sql 
select col as category, count(*) as count 
from 
(
  select explode(split(genres, '[|]')) 
  from movies
) 
group by 1
order by 2 desc

category,count
Drama,4361
Comedy,3756
Thriller,1894
Action,1828
Romance,1596
Adventure,1263
Crime,1199
Sci-Fi,980
Horror,978
Fantasy,779


Conclude movie titles under specific category listed above (Aggregate all movies into same category as a list)

In [30]:
%sql
select col as category, collect_list(title)
from 
(
select explode(split(genres, '[|]')), title
from movies
)
group by 1
order by 1

category collect_list(title) (no genres listed) List(La cravate (1957), Ben-hur (2016), Pirates of the Caribbean: Dead Men Tell No Tales (2017), Superfast! (2015), Let It Be Me (1995), Trevor Noah: African American (2013), Guardians (2016), Green Room (2015), The Brand New Testament (2015), Hyena Road, The Adventures of Sherlock Holmes and Doctor Watson, A Cosmic Christmas (1977), Grease Live (2016), Noin 7 veljestä (1968), Paterson, Ali Wong: Baby Cobra (2016), A Midsummer Night's Dream (2016), The Forbidden Dance (1990), Ethel & Ernest (2016), Whiplash (2013), The OA, Lemonade (2016), Cosmos, Maria Bamford: Old Baby, Death Note: Desu nôto (2006–2007), Generation Iron 2, T2 3-D: Battle Across Time (1996), The Godfather Trilogy: 1972-1990 (1992), The Adventures of Sherlock Holmes and Doctor Watson: The Hunt for the Tiger (1980), The Putin Interviews (2017), Black Mirror, Too Funny to Fail: The Life and Death of The Dana Carvey Show (2017), Serving in Silence: The Margarethe Cammermeyer Story (1995), A Christmas Story Live! (2017)) Action List(Heat (1995), Sudden Death (1995), GoldenEye (1995), Cutthroat Island (1995), Money Train (1995), Assassins (1995), Dead Presidents (1995), Mortal Kombat (1995), Lawnmower Man 2: Beyond Cyberspace (1996), From Dusk Till Dawn (1996), Fair Game (1995), Screamers (1995), Crossing Guard, The (1995), White Squall (1996), Nick of Time (1995), Broken Arrow (1996), Braveheart (1995), Rumble in the Bronx (Hont faan kui) (1995), Bad Boys (1995), Rob Roy (1995), Batman Forever (1995), Congo (1995), Desperado (1995), Die Hard: With a Vengeance (1995), First Knight (1995), Hackers (1995), Johnny Mnemonic (1995), Judge Dredd (1995), Mighty Morphin Power Rangers: The Movie (1995), Net, The (1995), Strange Days (1995), Under Siege 2: Dark Territory (1995), Waterworld (1995), Drop Zone (1994), French Kiss (1995), Hunted, The (1995), Star Wars: Episode IV - A New Hope (1977), New York Cop (Nyû Yôku no koppu) (1993), Natural Born Killers (1994), Outbreak (1995), Léon: The Professional (a.k.a. The Professional) (Léon) (1994), Quick and the Dead, The (1995), Specialist, The (1994), Stargate (1994), Tank Girl (1995), Tales from the Hood (1995), Virtuosity (1995), Clear and Present Danger (1994), Crow, The (1994), I Love Trouble (1994), Mask, The (1994), Naked Gun 33 1/3: The Final Insult (1994), River Wild, The (1994), Speed (1994), Timecop (1994), True Lies (1994), Bad Company (1995), Low Down Dirty Shame, A (1994), Faster Pussycat! Kill! Kill! (1965), Street Fighter (1994), Highlander III: The Sorcerer (a.k.a. Highlander: The Final Dimension) (1994), Beverly Hills Cop III (1994), Blown Away (1994), Cliffhanger (1993), Cowboy Way, The (1994), Demolition Man (1993), Getaway, The (1994), Hard Target (1993), Hot Shots! Part Deux (1993), In the Line of Fire (1993), Judgment Night (1993), Jurassic Park (1993), Last Action Hero (1993), Menace II Society (1993), Executive Decision (1996), Next Karate Kid, The (1994), No Escape (1994), Program, The (1993), Rising Sun (1993), RoboCop 3 (1993), Romper Stomper (1992), Shadow, The (1994), Blade Runner (1982), Striking Distance (1993), Super Mario Bros. (1993), Surviving the Game (1994), Terminal Velocity (1994), Three Musketeers, The (1993), Tombstone (1993), Pagemaster, The (1994), Terminator 2: Judgment Day (1991), Batman (1989), Love and a .45 (1994), Heavy Metal (1981), Hellraiser: Bloodline (1996), Courage Under Fire (1996), Mission: Impossible (1996), Dragonheart (1996), Bloodsport 2 (a.k.a. Bloodsport II: The Next Kumite) (1996), Operation Dumbo Drop (1995), Solo (1996), Substitute, The (1996), Quest, The (1996), Rock, The (1996), Twister (1996), Barb Wire (1996), Arrival, The (1996), Phantom, The (1996), Independence Day (a.k.a. ID4) (1996), Eraser (1996), Daylight (1996), Fled (1996), Chain Reaction (1996), Crow: City of Angels, The (1996), Escape from L.A. (1996), Supercop (Police Story 3: Supercop) (Jing cha gu shi III: Chao ji jing cha) (1992), Nothing to Lose

## Part 2: Spark ALS based approach for training model
We will use an RDD-based API from pyspark.mllib to predict the ratings, so let's reload "ratings.csv" using sc.textFile and then convert it to the form of (user, item, rating) tuples.

In [32]:
# change data type for table "ratings"
from pyspark.sql.types import IntegerType, FloatType
ratings = ratings.withColumn("userID", ratings["userID"].cast(IntegerType()))
ratings = ratings.withColumn("movieID", ratings["movieID"].cast(IntegerType()))
ratings = ratings.withColumn("rating", ratings["rating"].cast(FloatType()))
ratings.printSchema()

root
-- userID: integer (nullable = true)
-- movieID: integer (nullable = true)
-- rating: float (nullable = true)
-- timestamp: string (nullable = true)

In [33]:
ratings.show()

+------+-------+------+---------+
userID|movieID|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 70| 3.0|964982400|
 1| 101| 5.0|964980868|
 1| 110| 4.0|964982176|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 223| 3.0|964980985|
 1| 231| 5.0|964981179|
 1| 235| 4.0|964980908|
 1| 260| 5.0|964981680|
 1| 296| 3.0|964982967|
 1| 316| 3.0|964982310|
 1| 333| 5.0|964981179|
 1| 349| 4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows

In [34]:
ratings = ratings.drop('timestamp')

In [35]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [36]:
# train test spliting
(training, testing) = ratings.randomSplit([0.8, 0.2])

###ALS Model Selection (hyperparameter tuning) and Evaluation
With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [38]:
# build recommendation model using ALS on the training data
als = ALS(userCol="userID", itemCol="movieID", ratingCol="rating", coldStartStrategy="drop", nonnegative = True)

In [39]:
# use spark ML gridbuilder to find hyperparameter
paramGrid = ParamGridBuilder()\
.addGrid(als.rank, [8, 10, 12])\
.addGrid(als.maxIter, [10, 15, 20])\
.addGrid(als.regParam, [0.05, 0.1, 0.2])\
.build()

In [40]:
# Evaluate the model by computing the RMSE on the test data
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [41]:
# Hyperparameter Gridsearch Crossvalidation
# Hyperparameter Tuning with Gridsearch crossvalidation (2-fold)
# I originally used 3 fold crossvalidation but it takes too long to process.
# 2-fold cv is super slow as well..........
cv = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator= evaluator,
                          numFolds=2)  

In [42]:
model = cv.fit(training)

In [43]:
# get best model result from cross validation
best_model = model.bestModel

In [44]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(testing)
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.87916292662438

In [45]:
# print best model hyperparameters after testing and rmse value
# print('Best Param (Rank): '), best_model.rank
# print('Best Param (MaxIter): '), best_model._java_obj.getMaxIter()
# print('Best Param (regParam): '), best_model._java_obj.getRegParam()
# I googled and find sth on stackoverflow...didn't work as the example...why?
# https://stackoverflow.com/questions/36697304/how-to-extract-model-hyper-parameters-from-spark-ml-in-pyspark
print('Rmse =' + str(rmse))

Rmse =0.87916292662438

In [46]:
predictions.show()

+------+-------+------+----------+
userID|movieID|rating|prediction|
+------+-------+------+----------+
 133| 471| 4.0| 2.7861955|
 597| 471| 2.0| 3.9391198|
 385| 471| 4.0| 3.213863|
 436| 471| 3.0| 3.1545136|
 91| 471| 1.0| 3.1230369|
 409| 471| 3.0| 3.7579749|
 182| 471| 4.5| 3.6877012|
 216| 471| 3.0| 3.4125748|
 426| 471| 5.0| 3.2247975|
 104| 471| 4.5| 3.2825065|
 608| 833| 0.5| 2.0387197|
 159| 1088| 4.0| 3.0913708|
 111| 1088| 3.0| 3.384914|
 554| 1088| 5.0| 3.5375805|
 286| 1088| 3.5| 3.2511034|
 84| 1088| 3.0| 3.184686|
 51| 1088| 4.0| 3.326001|
 483| 1088| 3.0| 3.2724419|
 325| 1238| 4.0| 4.2549567|
 19| 1238| 3.0| 3.0794613|
+------+-------+------+----------+
only showing top 20 rows

### Apply model to all data

In [48]:
complete_data = best_model.transform(ratings)
rmse = evaluator.evaluate(complete_data)
print('Rmse =' + str(rmse))

Rmse =0.7464393802646389

In [49]:
complete_data.show()

+------+-------+------+----------+
userID|movieID|rating|prediction|
+------+-------+------+----------+
 191| 148| 5.0| 4.787312|
 133| 471| 4.0| 2.7861955|
 597| 471| 2.0| 3.9391198|
 385| 471| 4.0| 3.213863|
 436| 471| 3.0| 3.1545136|
 602| 471| 4.0| 3.1588104|
 91| 471| 1.0| 3.1230369|
 409| 471| 3.0| 3.7579749|
 372| 471| 3.0| 3.1748586|
 599| 471| 2.5| 2.8023527|
 603| 471| 4.0| 3.4883873|
 182| 471| 4.5| 3.6877012|
 218| 471| 4.0| 3.3025825|
 474| 471| 3.0| 3.342532|
 500| 471| 1.0| 2.7044811|
 57| 471| 3.0| 3.489973|
 462| 471| 2.5| 3.1784668|
 387| 471| 3.0| 3.2218895|
 610| 471| 4.0| 3.68375|
 217| 471| 2.0| 2.5623333|
+------+-------+------+----------+
only showing top 20 rows

In [50]:
# Generate top 10 movie recommendations for each user
userRecs = best_model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = best_model.recommendForAllItems(10)

In [51]:
display(userRecs)

userID,recommendations
471,"List(List(51931, 5.320682), List(68945, 4.922578), List(3379, 4.922578), List(5915, 4.7151494), List(132333, 4.7151494), List(33649, 4.66131), List(4495, 4.6337748), List(7842, 4.629262), List(92494, 4.6081843), List(102217, 4.6081843))"
463,"List(List(51931, 5.3267274), List(68945, 5.083486), List(3379, 5.083486), List(33649, 4.907832), List(7842, 4.9007907), List(132333, 4.852475), List(5915, 4.852475), List(141718, 4.7956), List(67618, 4.734514), List(4495, 4.727367))"
496,"List(List(51931, 4.973079), List(3379, 4.491978), List(68945, 4.491978), List(25947, 4.455985), List(26326, 4.433766), List(7008, 4.3867054), List(33649, 4.3787956), List(5915, 4.3219585), List(132333, 4.3219585), List(92494, 4.320496))"
148,"List(List(51931, 5.1659436), List(67618, 4.6016073), List(33649, 4.555925), List(6732, 4.513789), List(3379, 4.5098896), List(68945, 4.5098896), List(87234, 4.4125886), List(5915, 4.365093), List(132333, 4.365093), List(80586, 4.3649044))"
540,"List(List(51931, 5.875327), List(68945, 5.566249), List(3379, 5.566249), List(7842, 5.399293), List(33649, 5.382343), List(5915, 5.3367305), List(132333, 5.3367305), List(92494, 5.2125325), List(102217, 5.2125325), List(67618, 5.1213593))"
392,"List(List(51931, 5.0690928), List(68945, 4.6635833), List(3379, 4.6635833), List(4495, 4.5414586), List(141718, 4.4322643), List(136469, 4.425514), List(7071, 4.3762755), List(117531, 4.3762755), List(138966, 4.3762755), List(179135, 4.3762755))"
243,"List(List(51931, 5.9676924), List(67618, 5.728861), List(33649, 5.6515594), List(3379, 5.571946), List(68945, 5.571946), List(141718, 5.47536), List(7071, 5.468495), List(138966, 5.468495), List(117531, 5.468495), List(84273, 5.468495))"
31,"List(List(51931, 5.8145905), List(5915, 5.356616), List(132333, 5.356616), List(33649, 5.294708), List(67618, 5.27164), List(6732, 5.255296), List(68945, 5.250283), List(3379, 5.250283), List(4495, 5.096603), List(7842, 5.08202))"
516,"List(List(51931, 5.1621914), List(5915, 5.120839), List(132333, 5.120839), List(68945, 5.048148), List(3379, 5.048148), List(7842, 4.914357), List(33649, 4.859597), List(4495, 4.743396), List(152105, 4.6087546), List(5607, 4.6045))"
580,"List(List(51931, 5.195068), List(68945, 4.8289247), List(3379, 4.8289247), List(7842, 4.704274), List(33649, 4.676862), List(134796, 4.6251087), List(84273, 4.6251087), List(117531, 4.6251087), List(7071, 4.6251087), List(179135, 4.6251087))"


In [52]:
display(movieRecs)

movieID,recommendations
1580,"List(List(53, 4.8565636), List(543, 4.38772), List(43, 4.3679433), List(276, 4.356878), List(452, 4.332044), List(93, 4.3281116), List(12, 4.302332), List(523, 4.2209945), List(389, 4.1986804), List(25, 4.1836705))"
4900,"List(List(53, 4.749479), List(154, 4.2781453), List(236, 4.2381253), List(452, 4.183682), List(122, 4.130588), List(171, 4.1225233), List(441, 4.115959), List(523, 4.1152368), List(276, 4.1051216), List(12, 4.099917))"
5300,"List(List(53, 4.4552455), List(236, 4.2718005), List(171, 4.1093774), List(122, 4.069926), List(371, 4.0470076), List(348, 4.004311), List(515, 3.9813254), List(523, 3.9810247), List(25, 3.9637668), List(59, 3.9473403))"
6620,"List(List(236, 4.8887477), List(53, 4.6015368), List(296, 4.507552), List(168, 4.41467), List(122, 4.4060473), List(171, 4.389037), List(59, 4.385684), List(253, 4.3034177), List(418, 4.277481), List(258, 4.2706065))"
7340,"List(List(53, 4.3343954), List(543, 4.1031137), List(12, 4.0620146), List(43, 4.061173), List(276, 3.9089994), List(452, 3.8975837), List(93, 3.8043308), List(337, 3.7927616), List(558, 3.7733824), List(169, 3.7665405))"
32460,"List(List(53, 4.7872095), List(452, 4.2660227), List(236, 4.1951885), List(276, 4.1888914), List(523, 4.154351), List(122, 4.1526446), List(25, 4.1282578), List(515, 4.1053967), List(43, 4.101742), List(171, 4.0974426))"
54190,"List(List(53, 5.4493394), List(543, 4.9353247), List(43, 4.8777895), List(452, 4.8730087), List(276, 4.831444), List(12, 4.758771), List(523, 4.709535), List(93, 4.670612), List(25, 4.6372576), List(154, 4.6330338))"
471,"List(List(53, 4.5556345), List(236, 4.3387914), List(154, 4.275229), List(515, 4.21047), List(122, 4.2073617), List(171, 4.1947775), List(1, 4.1583953), List(348, 4.1473484), List(523, 4.1193132), List(25, 4.1161237))"
1591,"List(List(53, 3.8528793), List(236, 3.489319), List(452, 3.4179583), List(276, 3.3683553), List(537, 3.3402739), List(371, 3.307964), List(548, 3.2814121), List(441, 3.259598), List(122, 3.2515812), List(119, 3.2426376))"
140541,"List(List(43, 3.9042158), List(89, 3.8011768), List(543, 3.7385263), List(53, 3.70568), List(492, 3.6916385), List(558, 3.6664357), List(337, 3.5937114), List(240, 3.5697565), List(243, 3.5603905), List(578, 3.5397131))"


In [53]:
# Additional Question need further research:
# RMSE --- definition, math behind it
# Print out best model hyperparameters
# Netflix Dataset
# Can I join the movie titile to the result dataset? (join the tables before training?)

### Conclusion
In this project We used ALS algorithm to train on explicit rating data from different users to different films.

First, we explored the data set on number of users, movie titles and movie genres. During the data ETL, we used spark SQL to conclude number of movie rated and number of movies under different category.
The second part of the project is to construct the recommendation system using ALS (alternating least squares) algorithm.
We used grid search and 2-fold cross validation to do the model selection. Unfortunately, I was unable to print the hyperparameter used by the best model.
After applying the model to complete dataset, we obtained 10 recommended movies for each users and 10 recommended users for each movie.